In [ ]:
"""
================================================================================
MBG DATA - EXPLORATORY DATA ANALYSIS & DATA CLEANSING PIPELINE
================================================================================
Purpose: Comprehensive EDA and cleaning for MBG dataset
Author: Data Quality Pipeline
Date: 2025
================================================================================
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re
import os
import hashlib
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("="*80)
print("🔍 MBG DATA - EDA & CLEANSING PIPELINE")
print("="*80)
print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

# ============================================
# PART 1: DATA LOADING
# ============================================
print("\n" + "="*80)
print("📂 PART 1: DATA LOADING")
print("="*80)

# Mount Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive mounted successfully!")
except:
    print("⚠️  Running without Google Drive mount")

# Load data
data_path = '/content/drive/MyDrive/Magister SI TelU/Semester 1/Analisis Data dan Perusahaan/Tugas Besar/Progress Week 9/mbg_data.csv'

print(f"\n📁 Loading data from:")
print(f"   {data_path}")

try:
    df_raw = pd.read_csv(data_path, encoding='utf-8-sig')
    print(f"\n✅ Data loaded successfully!")
    print(f"   • Shape: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns")
    print(f"   • Memory: {df_raw.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
except Exception as e:
    print(f"\n❌ Error loading data: {str(e)}")
    raise

# Create backup
df = df_raw.copy()
print(f"\n✅ Working copy created for analysis")

# ============================================
# PART 2: BASIC INFORMATION
# ============================================
print("\n" + "="*80)
print("📋 PART 2: BASIC INFORMATION")
print("="*80)

print(f"\n1️⃣  Column Information:")
print("-" * 80)
for i, col in enumerate(df.columns, 1):
    dtype = df[col].dtype
    non_null = df[col].notna().sum()
    null_count = df[col].isna().sum()
    null_pct = (null_count / len(df) * 100)
    print(f"   {i:2d}. {col:25s} | {str(dtype):15s} | {non_null:6,}/{len(df):6,} filled ({100-null_pct:5.1f}%)")

print(f"\n2️⃣  Missing Values Summary:")
print("-" * 80)
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
has_missing = False

for col in df.columns:
    if missing[col] > 0:
        has_missing = True
        print(f"   ⚠️  {col:25s}: {missing[col]:6,} missing ({missing_pct[col]:5.2f}%)")

if not has_missing:
    print("   ✅ No missing values detected!")

print(f"\n3️⃣  Data Types Distribution:")
print("-" * 80)
dtype_counts = df.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"   • {dtype}: {count} columns")

print(f"\n4️⃣  Sample Data Preview:")
print("-" * 80)
print(df.head(3).to_string())

# ============================================
# PART 3: DUPLICATE ANALYSIS (DETAILED)
# ============================================
print("\n" + "="*80)
print("🔍 PART 3: DETAILED DUPLICATE ANALYSIS")
print("="*80)

# Create content hash for accurate duplicate detection
df['content_hash'] = df['content'].apply(
    lambda x: hashlib.md5(str(x).encode()).hexdigest() if pd.notna(x) else None
)

print("\n1️⃣  Duplicate Analysis by Different Criteria:")
print("-" * 80)

# URL duplicates
url_dup_count = df.duplicated(subset=['url'], keep=False).sum()
url_unique = df['url'].nunique()
print(f"   📌 URL Analysis:")
print(f"      • Total unique URLs: {url_unique:,}")
print(f"      • Duplicate URL entries: {url_dup_count:,}")
print(f"      • URL uniqueness: {url_unique/len(df)*100:.1f}%")

# Title duplicates
title_dup_count = df.duplicated(subset=['title'], keep=False).sum()
title_unique = df['title'].nunique()
print(f"\n   📌 Title Analysis:")
print(f"      • Total unique titles: {title_unique:,}")
print(f"      • Duplicate title entries: {title_dup_count:,}")
print(f"      • Title uniqueness: {title_unique/len(df)*100:.1f}%")

# Content duplicates
content_dup_count = df.duplicated(subset=['content_hash'], keep=False).sum()
content_unique = df['content_hash'].nunique()
print(f"\n   📌 Content Analysis:")
print(f"      • Total unique contents: {content_unique:,}")
print(f"      • Duplicate content entries: {content_dup_count:,}")
print(f"      • Content uniqueness: {content_unique/len(df)*100:.1f}%")

print("\n2️⃣  True Duplicates (Same Content):")
print("-" * 80)

true_duplicates = df[df.duplicated(subset=['content_hash'], keep=False)]
if len(true_duplicates) > 0:
    print(f"   ⚠️  Found {len(true_duplicates):,} rows with duplicate content")

    # Show examples
    content_counts = df['content_hash'].value_counts()
    multi_contents = content_counts[content_counts > 1].head(3)

    print(f"\n   📋 Top 3 Most Duplicated Contents:")
    for idx, (hash_val, count) in enumerate(multi_contents.items(), 1):
        articles = df[df['content_hash'] == hash_val][['title', 'date', 'category']].head(2)
        print(f"\n   Example {idx}: Appears {count}x")
        for i, row in articles.iterrows():
            print(f"      • {row['title'][:60]}...")
            print(f"        Date: {row['date']} | Category: {row['category']}")
else:
    print("   ✅ No duplicate content found!")

print("\n3️⃣  Duplicate Detection Conclusion:")
print("-" * 80)
print(f"   📊 Summary:")
print(f"      • Original dataset: {len(df):,} rows")
print(f"      • Unique by URL: {url_unique:,} rows ({url_unique/len(df)*100:.1f}%)")
print(f"      • Unique by Title: {title_unique:,} rows ({title_unique/len(df)*100:.1f}%)")
print(f"      • Unique by Content: {content_unique:,} rows ({content_unique/len(df)*100:.1f}%)")
print(f"\n   💡 Recommendation:")
print(f"      • Use CONTENT HASH for deduplication")
print(f"      • Expected clean data: ~{content_unique:,} unique articles")

# ============================================
# PART 4: CONTENT STATISTICS
# ============================================
print("\n" + "="*80)
print("📊 PART 4: CONTENT STATISTICS")
print("="*80)

# Calculate metrics
df['content_length'] = df['content'].fillna('').str.len()
df['word_count'] = df['content'].fillna('').str.split().str.len()
df['title_length'] = df['title'].fillna('').str.len()

print(f"\n1️⃣  Content Length Statistics:")
print("-" * 80)
print(f"   • Mean:     {df['content_length'].mean():,.0f} characters")
print(f"   • Median:   {df['content_length'].median():,.0f} characters")
print(f"   • Min:      {df['content_length'].min():,.0f} characters")
print(f"   • Max:      {df['content_length'].max():,.0f} characters")
print(f"   • Std Dev:  {df['content_length'].std():,.0f} characters")

print(f"\n2️⃣  Word Count Statistics:")
print("-" * 80)
print(f"   • Mean:     {df['word_count'].mean():,.0f} words")
print(f"   • Median:   {df['word_count'].median():,.0f} words")
print(f"   • Min:      {df['word_count'].min():,.0f} words")
print(f"   • Max:      {df['word_count'].max():,.0f} words")

print(f"\n3️⃣  Content Length Distribution:")
print("-" * 80)
bins = [0, 500, 1000, 2000, 3000, 5000, 10000]
labels = ['<500', '500-1K', '1K-2K', '2K-3K', '3K-5K', '>5K']
df['content_category'] = pd.cut(df['content_length'], bins=bins, labels=labels)

dist = df['content_category'].value_counts().sort_index()
for cat, count in dist.items():
    pct = count / len(df) * 100
    bar = '█' * int(pct / 2)
    print(f"   {cat:10s}: {count:5,} articles ({pct:5.1f}%) {bar}")

# ============================================
# PART 5: CATEGORY ANALYSIS
# ============================================
print("\n" + "="*80)
print("🏷️  PART 5: CATEGORY ANALYSIS")
print("="*80)

print(f"\n1️⃣  Category Distribution:")
print("-" * 80)
cat_counts = df['category'].value_counts()
total_articles = len(df)

for idx, (cat, count) in enumerate(cat_counts.items(), 1):
    pct = count / total_articles * 100
    bar = '█' * int(pct / 2)
    print(f"   {idx:2d}. {cat:30s}: {count:5,} ({pct:5.1f}%) {bar}")

print(f"\n2️⃣  Category Statistics:")
print("-" * 80)
print(f"   • Total unique categories: {df['category'].nunique()}")
print(f"   • Most common: {cat_counts.index[0]} ({cat_counts.values[0]:,} articles)")
print(f"   • Least common: {cat_counts.index[-1]} ({cat_counts.values[-1]:,} articles)")
print(f"   • Articles without category: {df['category'].isna().sum()}")

# ============================================
# PART 6: AUTHOR ANALYSIS
# ============================================
print("\n" + "="*80)
print("✍️  PART 6: AUTHOR ANALYSIS")
print("="*80)

author_counts = df['author'].fillna('Unknown').value_counts()

print(f"\n1️⃣  Top 15 Most Productive Authors:")
print("-" * 80)
for idx, (author, count) in enumerate(author_counts.head(15).items(), 1):
    pct = count / len(df) * 100
    print(f"   {idx:2d}. {author:40s}: {count:4,} articles ({pct:5.1f}%)")

print(f"\n2️⃣  Author Statistics:")
print("-" * 80)
print(f"   • Total unique authors: {df['author'].nunique():,}")
print(f"   • Articles without author: {df['author'].isna().sum():,}")
print(f"   • Average articles per author: {len(df) / df['author'].nunique():.1f}")

# ============================================
# PART 7: DATE ANALYSIS
# ============================================
print("\n" + "="*80)
print("📅 PART 7: DATE ANALYSIS")
print("="*80)

print(f"\n1️⃣  Date Format Examples:")
print("-" * 80)
for i, date in enumerate(df['date'].dropna().head(5), 1):
    print(f"   {i}. {date}")

print(f"\n2️⃣  Date Completeness:")
print("-" * 80)
print(f"   • Articles with date: {df['date'].notna().sum():,} ({df['date'].notna().sum()/len(df)*100:.1f}%)")
print(f"   • Articles without date: {df['date'].isna().sum():,} ({df['date'].isna().sum()/len(df)*100:.1f}%)")

# Extract year from date string
df['year_extracted'] = df['date'].str.extract(r'(\d{4})')

if df['year_extracted'].notna().sum() > 0:
    year_counts = df['year_extracted'].value_counts().sort_index()
    print(f"\n3️⃣  Articles by Year:")
    print("-" * 80)
    for year, count in year_counts.items():
        pct = count / len(df) * 100
        bar = '█' * int(pct / 2)
        print(f"   {year}: {count:5,} articles ({pct:5.1f}%) {bar}")

# ============================================
# PART 8: MBG-SPECIFIC CONTENT ANALYSIS
# ============================================
print("\n" + "="*80)
print("🍽️  PART 8: MBG-SPECIFIC CONTENT ANALYSIS")
print("="*80)

# Keywords analysis
dist_keywords = ['distribusi', 'pembagian', 'menyalurkan', 'membagikan', 'bantuan', 'porsi']
df['contains_distribution'] = df['content'].fillna('').str.lower().str.contains('|'.join(dist_keywords))

print(f"\n1️⃣  Distribution Content Detection:")
print("-" * 80)
dist_count = df['contains_distribution'].sum()
print(f"   • Articles about distribution: {dist_count:,}/{len(df):,} ({dist_count/len(df)*100:.1f}%)")

# Number patterns
porsi_pattern = r'(\d+[\.,]?\d*)\s*(?:porsi|makanan|paket)'
penerima_pattern = r'(\d+[\.,]?\d*)\s*(?:orang|siswa|anak|penerima|warga)'

df['has_porsi_info'] = df['content'].str.contains(porsi_pattern, case=False, regex=True, na=False)
df['has_penerima_info'] = df['content'].str.contains(penerima_pattern, case=False, regex=True, na=False)

print(f"\n2️⃣  Quantitative Information:")
print("-" * 80)
print(f"   • Articles with portion info: {df['has_porsi_info'].sum():,}")
print(f"   • Articles with recipient info: {df['has_penerima_info'].sum():,}")

complete_info = df[
    df['contains_distribution'] &
    df['has_porsi_info'] &
    df['has_penerima_info']
]
print(f"   • Articles with complete info: {len(complete_info):,}")

# ============================================
# PART 9: DATA CLEANSING
# ============================================
print("\n" + "="*80)
print("🧹 PART 9: DATA CLEANSING")
print("="*80)

print(f"\n📊 Before Cleaning:")
print("-" * 80)
print(f"   • Total rows: {len(df):,}")
print(f"   • Unique content: {df['content_hash'].nunique():,}")
print(f"   • Duplicates to remove: {len(df) - df['content_hash'].nunique():,}")

# Remove duplicates based on content hash
df_clean = df.drop_duplicates(subset=['content_hash'], keep='first').copy()

print(f"\n1️⃣  Duplicate Removal:")
print("-" * 80)
print(f"   ✅ Removed {len(df) - len(df_clean):,} duplicate articles")
print(f"   ✅ Retained {len(df_clean):,} unique articles")

# Remove temporary columns
columns_to_drop = ['content_hash', 'content_category', 'year_extracted',
                   'contains_distribution', 'has_porsi_info', 'has_penerima_info']
df_clean = df_clean.drop(columns=[col for col in columns_to_drop if col in df_clean.columns])

print(f"\n2️⃣  Column Cleanup:")
print("-" * 80)
print(f"   ✅ Removed temporary analysis columns")
print(f"   ✅ Final columns: {len(df_clean.columns)}")

# Handle missing values (optional - keeping as is for now)
print(f"\n3️⃣  Missing Values Handling:")
print("-" * 80)
for col in df_clean.columns:
    missing = df_clean[col].isna().sum()
    if missing > 0:
        print(f"   • {col}: {missing:,} missing values (kept as is)")

print(f"\n📊 After Cleaning:")
print("-" * 80)
print(f"   • Total rows: {len(df_clean):,}")
print(f"   • Total columns: {len(df_clean.columns)}")
print(f"   • Memory usage: {df_clean.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")
print(f"   • Reduction: {(1 - len(df_clean)/len(df))*100:.1f}%")

# ============================================
# PART 10: SAVE CLEANED DATA
# ============================================
print("\n" + "="*80)
print("💾 PART 10: SAVING CLEANED DATA")
print("="*80)

# Define save locations
output_dir = '/content/drive/MyDrive/Magister SI TelU/Semester 1/Analisis Data dan Perusahaan/Tugas Besar/Progress Week 9/'
output_file = 'mbg_data_clean.csv'
output_path = os.path.join(output_dir, output_file)

# Save to Google Drive
try:
    df_clean.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"\n✅ Data saved to Google Drive:")
    print(f"   📁 {output_path}")
    print(f"   📊 Size: {os.path.getsize(output_path) / 1024 / 1024:.2f} MB")
except Exception as e:
    print(f"\n⚠️  Could not save to Google Drive: {str(e)}")

# Also save to local for immediate use
local_path = '/content/mbg_data_clean.csv'
try:
    df_clean.to_csv(local_path, index=False, encoding='utf-8-sig')
    print(f"\n✅ Data also saved locally:")
    print(f"   📁 {local_path}")
except Exception as e:
    print(f"\n⚠️  Could not save locally: {str(e)}")

# ============================================
# PART 11: FINAL SUMMARY REPORT
# ============================================
print("\n" + "="*80)
print("📋 PART 11: FINAL SUMMARY REPORT")
print("="*80)

print(f"\n🎯 EDA & CLEANSING SUMMARY:")
print("="*80)

print(f"\n📊 Dataset Transformation:")
print(f"   • Original dataset: {len(df):,} articles")
print(f"   • Cleaned dataset: {len(df_clean):,} articles")
print(f"   • Removed duplicates: {len(df) - len(df_clean):,} articles")
print(f"   • Data reduction: {(1 - len(df_clean)/len(df))*100:.1f}%")

print(f"\n🔍 Data Quality Metrics:")
print(f"   • Content uniqueness: 100.0%")
print(f"   • Date completeness: {df_clean['date'].notna().sum()/len(df_clean)*100:.1f}%")
print(f"   • Author completeness: {df_clean['author'].notna().sum()/len(df_clean)*100:.1f}%")
print(f"   • Category completeness: {df_clean['category'].notna().sum()/len(df_clean)*100:.1f}%")

print(f"\n📈 Content Characteristics:")
print(f"   • Unique categories: {df_clean['category'].nunique()}")
print(f"   • Unique authors: {df_clean['author'].nunique()}")
print(f"   • Avg content length: {df_clean['content_length'].mean():,.0f} chars")
print(f"   • Avg word count: {df_clean['word_count'].mean():,.0f} words")

print(f"\n✅ Quality Assurance:")
print(f"   ✓ No duplicate content")
print(f"   ✓ All columns preserved")
print(f"   ✓ Data integrity maintained")
print(f"   ✓ Ready for Soda Core validation")

print(f"\n🎯 Next Steps:")
print(f"   1. ✅ Data is ready at: {output_file}")
print(f"   2. 📋 Use this file for Soda Core setup")
print(f"   3. 🔍 Define data quality checks")
print(f"   4. 📊 Monitor data quality metrics")

print("\n" + "="*80)
print("✅ EDA & CLEANSING COMPLETED SUCCESSFULLY!")
print("="*80)
print(f"Finished at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

print(f"\n💡 IMPORTANT NOTES:")
print(f"   • Cleaned data saved to Google Drive and local")
print(f"   • Use 'mbg_data_clean.csv' for Soda Core")
print(f"   • All duplicates removed based on content hash")
print(f"   • Data quality significantly improved")
print("="*80)

🔍 MBG DATA - EDA & CLEANSING PIPELINE
Started at: 2025-11-24 15:52:06

📂 PART 1: DATA LOADING
Mounted at /content/drive
✅ Google Drive mounted successfully!

📁 Loading data from:
   /content/drive/MyDrive/Magister SI TelU/Semester 1/Analisis Data dan Perusahaan/Tugas Besar/Progress Week 9/mbg_data.csv

✅ Data loaded successfully!
   • Shape: 244 rows × 10 columns
   • Memory: 1.08 MB

✅ Working copy created for analysis

📋 PART 2: BASIC INFORMATION

1️⃣  Column Information:
--------------------------------------------------------------------------------
    1. url                       | object          |    244/   244 filled (100.0%)
    2. title                     | object          |    244/   244 filled (100.0%)
    3. date                      | object          |    244/   244 filled (100.0%)
    4. author                    | object          |    234/   244 filled ( 95.9%)
    5. category                  | object          |    244/   244 filled (100.0%)
    6. content           